In [3]:
pip install inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.6/888.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 53.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.

In [ ]:
pip install mediapipe

In [ ]:
from inference import get_model
import supervision as sv
import cv2
import numpy as np
from matplotlib import pyplot as plt
# import preprocessing as prep
import cv2
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
# from sklearn.neighbors import NearestNeighbors
# import imutils
# import math
# from sklearn.cluster import DBSCAN
# from skimage.morphology import skeletonize
import time
# from ultralytics import YOLO
# from ultralytics.utils.plotting import Annotator
import scipy.spatial as spatial
from statistics import mean
import scipy.cluster as cluster
# from inference import get_model
from PIL import Image, ImageChops
# from imutils.object_detection import non_max_suppression
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
import torch
# import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# import eminst
# import tensorflow as tf
import mediapipe as mp

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

In [ ]:
def augment_points(points):
    points_shape = list(np.shape(points))
    augmented_points = []
    for row in range(int(points_shape[0] / 11)):
        start = row * 11
        end = (row * 11) + 10
        rw_points = points[start:end + 1]
        rw_y = []
        rw_x = []
        for point in rw_points:
            x, y = point
            rw_y.append(y)
            rw_x.append(x)
        y_mean = mean(rw_y)
        for i in range(len(rw_x)):
            point = (rw_x[i], y_mean)
            augmented_points.append(point)
    augmented_points = sorted(augmented_points, key=lambda k: [k[1], k[0]])
    return augmented_points


In [ ]:
def intersection_of_line(line1, line2):
    rho1, theta1 = line1
    rho2, theta2 = line2
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [x0, y0]


def find_intersections_point(vertical_lines, horizontal_lines):
    intersections = []
    for v_line in vertical_lines:
        for h_line in horizontal_lines:
            intersections.append(intersection_of_line(v_line, h_line))
    return intersections

In [ ]:

def get_board_mask(image, model, threshold=0, expansion_factor=0):
    # Create an empty mask of the same size as the input image
    mask = np.zeros_like(image, dtype=np.uint8)
    
    # Get the results from the model
    results = model(image)
    
    # Initialize edge variable
    edge = None
    
    for result in results:
        for box in result.boxes:
            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(image.shape[1], x2)
            y2 = min(image.shape[0], y2)
            
            # Compute the length of the bounding box diagonal (edge length)
            edge_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))  
            
            # If the edge length is above the threshold, process it
            if edge_length >= threshold:
                # Expand the bounding box based on the threshold
                x1_expanded = max(0, x1 - expansion_factor)
                y1_expanded = max(0, y1 - expansion_factor)
                x2_expanded = min(image.shape[1], x2 + expansion_factor)
                y2_expanded = min(image.shape[0], y2 + expansion_factor)

                # Create the expanded bounding box points
                top_left = (x1_expanded, y1_expanded)
                bottom_right = (x2_expanded, y2_expanded)
                top_right = (x2_expanded, y1_expanded)
                bottom_left = (x1_expanded, y2_expanded)

                # Define the expanded polygon and fill the mask
                points = np.array([[top_left, top_right, bottom_right, bottom_left]], dtype=np.int32)
                cv2.fillPoly(mask, points, (255, 255, 255))
                
                # Store the edge (expanded bounding box coordinates) for return
                edge = np.array([[x1_expanded, y1_expanded], [x2_expanded, y1_expanded], 
                                 [x1_expanded, y2_expanded], [x2_expanded, y2_expanded]], dtype=np.float32)
    return mask, edge

def categorize_hough_lines(lines, angle_threshold=np.radians(10)):
        vertical_lines = []
        horizontal_lines = []
        for rho, theta in lines:
            theta = theta % np.pi
            if abs(theta - np.pi/2) < angle_threshold:
                vertical_lines.append((rho, theta))
            elif theta < angle_threshold or abs(theta - np.pi) < angle_threshold:
                horizontal_lines.append((rho, theta))
        return vertical_lines, horizontal_lines

def cluster_intersections(points, threshold):
    clusters = []
    visited = [False] * len(points)  
    for i, point1 in enumerate(points):
        if visited[i]:
            continue
        cluster = [point1]
        visited[i] = True
        for j, point2 in enumerate(points):
            if not visited[j] and euclidean_distance(point1, point2) < threshold:
                cluster.append(point2)
                visited[j] = True
        centroid_x = np.mean([p[0] for p in cluster])
        centroid_y = np.mean([p[1] for p in cluster])
        clusters.append((centroid_x, centroid_y))  
    return clusters
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def crop_board(image,mask):
    return cv2.bitwise_and(image,mask)

def get_grid(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  
    gray = cv2.GaussianBlur(gray,(5,5),0)
    edges = cv2.Canny(gray, 30, 100)
    
    height, width = image.shape[:2]

    grid = [[[] for _ in range(8)] for _ in range(8)]

    grid_visualization = np.zeros_like(gray)
    
    image = cv2.bitwise_not(grid_visualization)
    edge_width = int(0.01 * min(height, width))
    mask = np.zeros((height, width), dtype=np.uint8)
    top = edge_width
    left = edge_width
    bottom = height - edge_width
    right = width - edge_width

    mask[top:bottom, left:right] = 1

    edges = cv2.bitwise_and(edges,mask)
    
    
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 130)

    
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            epsilon = 0.1  
            if abs(theta) < epsilon or abs(theta - np.pi) < epsilon or abs(theta - np.pi / 2) < epsilon:
                x1 = int(rho * np.cos(theta) + 1000 * (-np.sin(theta)))
                y1 = int(rho * np.sin(theta) + 1000 * (np.cos(theta)))
                x2 = int(rho * np.cos(theta) - 1000 * (-np.sin(theta)))
                y2 = int(rho * np.sin(theta) - 1000 * (np.cos(theta)))
                cv2.line(grid_visualization, (x1, y1), (x2, y2), 255,2)
    
    grid_visualization = cv2.bitwise_not(grid_visualization)

    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(grid_visualization, connectivity=8)
    
    # Filter and process centroids
    centroid_list = [
        (centroids[i][0], centroids[i][1], i, stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]) 
        for i in range(1, num_labels) 
        if stats[i, cv2.CC_STAT_AREA] > 2500 
    ]  
    

    if(len(centroid_list) !=  64): 
        print("Invalid Grid Initialization")
        return False,grid
    
    # Sort centroids
    centroid_list.sort(key=lambda x: x[1])  
    
    # Organize centroids into rows
    rows = [[] for _ in range(8)]
    for i, (cx, cy, label, width, height) in enumerate(centroid_list):
        row = i // 8  
        if len(rows) <= row:
            rows.extend([[] for _ in range(row - len(rows) + 1)])  
        rows[row].append((cx, cy, i + 1, label, width, height))
            
    # Sort each row by x-coordinate
    for row in rows:
        row.sort(key=lambda x: x[0])  
    
    # Process grid with dimensional information
    for i, row in enumerate(rows):
        for j, (cx, cy, idx, label, width, height) in enumerate(row):
            # Calculate width and height based on neighbors
            right_neighbor = row[j + 1] if j + 1 < len(row) else None
            below_neighbor = rows[i + 1][j] if i + 1 < len(rows) and j < len(rows[i + 1]) else None
            
            computed_width = (right_neighbor[0] - cx) if right_neighbor else width
            computed_height = (below_neighbor[1] - cy) if below_neighbor else height
            
            # Update grid matrix with (x, y, width, height)
            if i < 8 and j < 8:
                grid[i][j] = {
                    'x': cx,
                    'y': cy,
                    'width': computed_width,
                    'height': computed_height
                }
                
    return True,grid
    

    
def get_board_coordination(grid):
    coordinate = []
    return coordinate

    

def get_board_corner(image, model=None, distance_threshold=30):

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    close = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel1)
    
    blurred = cv2.GaussianBlur(close, (3, 3), 0)
    edges = cv2.Canny(blurred, 50, 150)
   
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    
    horizontal_lines = []
    vertical_lines = []
    
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            
            if (theta < np.pi / 4 or theta > 3 * np.pi / 4): 
                vertical_lines.append((rho, theta))
            else:  
                horizontal_lines.append((rho, theta))
    
    
    intersections_points = find_intersections_point(vertical_lines, horizontal_lines)
    
    if not intersections_points or len(intersections_points) < 4:
        raise ValueError("Not enough intersection points found to determine grid corners")
    
    
    
    top_left = min(intersections_points, key=lambda p: p[0] + p[1])
    
    top_right = max(intersections_points, key=lambda p: p[0] - p[1])
    
    bottom_left = max(intersections_points, key=lambda p: p[1] - p[0])
    
    bottom_right = max(intersections_points, key=lambda p: p[0] + p[1])
    
    corners = [(top_left), (top_right), (bottom_right),(bottom_left)]
    corners = np.array(corners, dtype=np.float32)
    
    return  corners



def polygon_area(points):
    points = np.array(points)
    x = points[:, 0]
    y = points[:, 1]
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    return area

def find_rectangle_by_point(points, num_points=4):
    rectangles = []
    for i, point in enumerate(points):
        x, y = point
        distances = [] 
        for j, other_point in enumerate(points):
            if i != j: 
                dist = euclidean_distance(point, other_point)
                distances.append((dist, other_point))
        distances.sort(key=lambda x: x[0]) 
        closest_points = [p for _, p in distances[:num_points]]  
        rectangle = [point] + closest_points
        rectangles.append(rectangle)
    return rectangles



def calculate_centroids(clusters):
    centroids = []
    for cluster in clusters:
        cluster_array = np.array(cluster)  
        centroid = cluster_array.mean(axis=0) 
        centroids.append(centroid.tolist())
    return centroids


def h_v_lines(lines):
    h_lines, v_lines = [], []
    for rho, theta in lines[:,0]:
        if theta < np.pi / 4 or theta > np.pi - np.pi / 4:
            v_lines.append([rho, theta])
        else:
            h_lines.append([rho, theta])
    return h_lines, v_lines

def line_intersections(h_lines, v_lines):
    points = []
    for r_h, t_h in h_lines:
        for r_v, t_v in v_lines:
            a = np.array([[np.cos(t_h), np.sin(t_h)], [np.cos(t_v), np.sin(t_v)]])
            b = np.array([r_h, r_v])
            inter_point = np.linalg.solve(a, b)
            points.append(inter_point)
    return np.array(points)

def calculate_cluster_points(points):
    dists = spatial.distance.pdist(points)
    single_linkage = cluster.hierarchy.single(dists)
    flat_clusters = cluster.hierarchy.fcluster(single_linkage, 15, 'distance')
    cluster_dict = defaultdict(list)
    for i in range(len(flat_clusters)):
        cluster_dict[flat_clusters[i]].append(points[i])
    cluster_values = cluster_dict.values()
    clusters = map(lambda arr: (np.mean(np.array(arr)[:, 0]), np.mean(np.array(arr)[:, 1])), cluster_values)
    return sorted(list(clusters), key=lambda k: [k[1], k[0]])
    

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def get_board_line(image, vertical_length=20, horizontal_length=20):
    canny = cv2.Canny(image,50,150)
    lines = cv2.HoughLines(canny, 1, np.pi / 180, 200) 
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * a)
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * a)
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

def test_board(image, model):
    results = model(image,conf=0.1)
    annotated_image = results[0].plot()
    return annotated_image

def trim(image):
    im = Image.fromarray(image)
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return np.asarray(im.crop(bbox))    
    return im
    
def apply_perspective_transform(image,original_point):
    destination = np.float32([[0, 0], [640, 0], [640, 640], [0, 640]])
    matrix = cv2.getPerspectiveTransform(original_point, destination)
    warp = cv2.warpPerspective(image, matrix, (640, 640))
    return warp,matrix

def call_pretrain(image, model):
    results = model.infer(image)[0] 
    predictions = results.predictions
    bounding_boxes = [] 
    for pred in predictions:
        x = int(pred.x)
        y = int(pred.y)
        width = int(pred.width)
        height = int(pred.height)
        confident = pred.confidence
        class_name = pred.class_name
        bbox_dict = {
            'class_name': class_name,
            'x': x,
            'y': y,
            'width': width,
            'height': height,
            'confidence': confident,
        }
        bounding_boxes.append(bbox_dict)
    
    return bounding_boxes




def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened_image = cv2.filter2D(image, -1, kernel)
    return sharpened_image

def get_chess_prediction(image,model):
    results = call_pretrain(image,model)
    return results
    
def get_annotated_image(image, bounding_boxes):
    annotated_image = image.copy()
    
    for bbox in bounding_boxes:
        x = bbox['x']
        y = bbox['y']
        width = bbox['width']
        height = bbox['height']
        confidence = bbox['confidence']
        class_name = bbox['class_name']
        x1 = int(x - width // 2)
        y1 = int(y - height // 2)
        x2 = int(x + width // 2)
        y2 = int(y + height // 2)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        text = f"{class_name}: {confidence:.2f}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.4
        font_thickness = 1
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = x1
        text_y = y1 - 10
        if text_y < 0:
            text_y = y1 + 10
        cv2.rectangle(annotated_image, 
                      (text_x, text_y - text_size[1] - 5), 
                      (text_x + text_size[0], text_y), 
                      (0, 255, 0), 
                      -1)
        cv2.putText(annotated_image, text, (text_x, text_y), 
                    font, font_scale, (0, 0, 0), font_thickness)
    return annotated_image

def transform_to_original_grid(perspective_grid, M_inv):
    # Create a new list to store the transformed grid
    transformed_grid = [[None for _ in range(len(perspective_grid[0]))] for _ in range(len(perspective_grid))]
    
    for i in range(len(perspective_grid)):
        for j in range(len(perspective_grid[i])):
            if perspective_grid[i][j] is not None:
                cell = perspective_grid[i][j]
                # print(f"Cell at [{i}][{j}]: {cell}")  # Detailed print to check the structure
                if isinstance(cell, dict):  # Ensure that the cell is a dictionary
                    x, y, width, height = cell['x'], cell['y'], cell['width'], cell['height']
                    pt1 = np.array([x, y, 1]).reshape(3, 1) 
                    transformed_pt1 = np.dot(M_inv, pt1)
                    transformed_pt1 /= transformed_pt1[2]  
                    transformed_x, transformed_y = transformed_pt1[0], transformed_pt1[1]
                    pt2 = np.array([x + width, y + height, 1]).reshape(3, 1)
                    transformed_pt2 = np.dot(M_inv, pt2)
                    transformed_pt2 /= transformed_pt2[2]
                    transformed_x2, transformed_y2 = transformed_pt2[0], transformed_pt2[1]
                    transformed_width = transformed_x2 - transformed_x
                    transformed_height = transformed_y2 - transformed_y
                    transformed_grid[i][j] = {
                        'x': transformed_x,
                        'y': transformed_y,
                        'width': transformed_width,
                        'height': transformed_height
                    }
                    
    return transformed_grid


def transform_prediction_to_perspective(predictions, perspective_matrix):
    transformed_predictions = []

    for prediction in predictions:
        class_name = prediction['class_name']
        x, y = prediction['x'], prediction['y']
        width, height = prediction['width'], prediction['height']
        confidence = prediction['confidence']
        
        x1, y1 = x - width / 2, y - height / 2
        x2, y2 = x + width / 2, y + height / 2

        pt1 = np.array([x1, y1, 1]).reshape(3, 1)
        pt2 = np.array([x2, y2, 1]).reshape(3, 1)

        transformed_pt1 = np.dot(perspective_matrix, pt1)
        transformed_pt2 = np.dot(perspective_matrix, pt2)

        transformed_pt1 /= transformed_pt1[2]
        transformed_pt2 /= transformed_pt2[2]

        transformed_x1, transformed_y1 = transformed_pt1[0], transformed_pt1[1]
        transformed_x2, transformed_y2 = transformed_pt2[0], transformed_pt2[1]

        transformed_width = transformed_x2 - transformed_x1
        transformed_height = transformed_y2 - transformed_y1

        transformed_x = transformed_x1 + transformed_width / 2
        transformed_y = transformed_y1 + transformed_height / 2

        transformed_prediction = {
            'class_name': class_name,
            'x': transformed_x,
            'y': transformed_y,
            'width': transformed_width,
            'height': transformed_height,
            'confidence': confidence
        }

        transformed_predictions.append(transformed_prediction)

    return transformed_predictions    


        
            
            
def remove_objects_and_crop(image, min_area=1000):
    image_area = image.shape[0] * image.shape[1]
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones(image.shape, dtype=np.uint8) * 255 
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area: 
            cv2.drawContours(mask, [contour], -1, (0, 0, 0), thickness=cv2.FILLED)
    result = cv2.bitwise_and(image, mask)
    gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    ret, binary_result = cv2.threshold(gray_result, 1, 255, cv2.THRESH_BINARY)
    x, y, w, h = cv2.boundingRect(binary_result)
    cropped_image = result[y:y+h, x:x+w]
    return cropped_image

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Define layers: 1 input layer (784 input features), 2 hidden layers, 1 output layer (10 classes)
        self.fc1 = nn.Linear(28 * 28, 128)  # 28x28 images flattened to 784 features
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 output classes (digits 0-9)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image (batch_size, 28 * 28)
        x = F.relu(self.fc1(x))   # Apply ReLU activation after first layer
        x = F.relu(self.fc2(x))   # Apply ReLU activation after second layer
        x = self.fc3(x)           # Output layer (no activation function for multi-class classification)
        return x
    
def train_mnist(model, train_loader, test_loader, epochs=5, lr=0.001):
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()  # Zero the gradients before each batch
            output = model(data)   # Forward pass
            loss = criterion(output, target)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters

            running_loss += loss.item()  # Accumulate loss
            _, predicted = torch.max(output.data, 1)  # Get predicted labels
            total += target.size(0)  # Total number of images
            correct += (predicted == target).sum().item()  # Correct predictions

        # Print statistics for the epoch
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

        # Evaluate on test data after every epoch
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():  # No need to compute gradients for evaluation
            for data, target in test_loader:
                output = model(data)  # Forward pass
                loss = criterion(output, target)  # Calculate loss
                test_loss += loss.item()  # Accumulate loss
                _, predicted = torch.max(output.data, 1)  # Get predicted labels
                total += target.size(0)  # Total number of images
                correct += (predicted == target).sum().item()  # Correct predictions

        print(f'Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%')

def train_minst():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

    model = SimpleNN()

    train_mnist(model, train_loader, test_loader, epochs=5, lr=0.001)
    return model

In [ ]:
def find_largest_component(image):
    # Ensure the input image is binary, no need for thresholding
    binary = image

    # Find connected components and their stats
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, 8, cv2.CV_32S)

    # Find the index of the largest component by area (ignore the background component with label 0)
    largest_component_index = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])  # Ignore background (label 0)

    # Get the bounding box of the largest component
    x, y, w, h, area = stats[largest_component_index]

    # Crop the image using the bounding box of the largest connected component
    cropped_image = image[y:y+h, x:x+w]

    return cropped_image

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
def find_largest_component(image):
    # Ensure the input image is binary, no need for thresholding
    binary = image

    # Find connected components and their stats
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, 8, cv2.CV_32S)

    # Find the index of the largest component by area (ignore the background component with label 0)
    largest_component_index = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])  # Ignore background (label 0)

    # Get the bounding box of the largest component
    x, y, w, h, area = stats[largest_component_index]

    # Crop the image using the bounding box of the largest connected component
    cropped_image = image[y:y+h, x:x+w]

    return cropped_image

In [ ]:
def calculate_trend_score(sequence):
    sequence = np.array(sequence.copy())
    differences = np.diff(sequence)
    trend_score = np.sum(np.sign(differences))
    return trend_score
  
def determine_trend(score):
    if score > 0:
        return "Increasing"
    else:
        return "Decreasing"

In [ ]:
def find_cell_color(image, alpha=2):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the grayscale image to create a binary image
    ret, binary_thresholded_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    
    # Calculate the proportion of black pixels
    black_pixels = np.sum(binary_thresholded_image == 0)
    total_pixels = binary_thresholded_image.size
    black_probability = black_pixels / total_pixels
    
    # If black pixels are over 80%, classify as black
    if black_probability > 0.8:
        return "black"
    
    # If black pixels are less than 20%, classify as white
    elif black_probability < 0.2:
        return "white"
    
    # Otherwise, perform the weighted calculation and classify based on the distance
    height, width = gray_image.shape
    center_x, center_y = width // 2, height // 2
    
    # Initialize an array to store the weights for each pixel
    weights = np.zeros_like(gray_image, dtype=np.float32)
    
    # Calculate the weights based on distance from the center
    for i in range(height):
        for j in range(width):
            distance = np.sqrt((i - center_y)**2 + (j - center_x)**2)
            weights[i, j] = (distance + 1) ** alpha  # Greater distances get exponentially larger weights
    
    # Normalize the weights to sum to 1
    weights /= np.sum(weights)
    
    # Calculate the weighted sum of the white pixel colors (if they are 255)
    weighted_sum = 0
    for i in range(height):
        for j in range(width):
            if binary_thresholded_image[i, j] == 255:  # White pixel in binary image
                weighted_sum += weights[i, j]
    # print(weighted_sum)
    # Classify based on the weighted sum
    if weighted_sum > 0.5:  # If white dominates, classify as white
        return "white"
    else:
        return "black"

In [ ]:
Board1 = [    ['a8', 'b8', 'c8', 'd8', 'e8', 'f8', 'g8', 'h8'],
    ['a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7'],
    ['a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6'],
    ['a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5', 'h5'],
    ['a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4'],
    ['a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3'],
    ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2'],
    ['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1']
]
#vertical:let,dec horizontal:num,asc
Board2 = [    ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8'],
    ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8'],
    ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8'],
    ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'],
    ['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8'],
    ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'],
    ['g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8'],
    ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8']
]
#vertical:num,dec horizontal:let,dec
Board3 = [    ['h1', 'g1', 'f1', 'e1', 'd1', 'c1', 'b1', 'a1'],
    ['h2', 'g2', 'f2', 'e2', 'd2', 'c2', 'b2', 'a2'],
    ['h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3'],
    ['h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4'],
    ['h5', 'g5', 'f5', 'e5', 'd5', 'c5', 'b5', 'a5'],
    ['h6', 'g6', 'f6', 'e6', 'd6', 'c6', 'b6', 'a6'],
    ['h7', 'g7', 'f7', 'e7', 'd7', 'c7', 'b7', 'a7'],
    ['h8', 'g8', 'f8', 'e8', 'd8', 'c8', 'b8', 'a8']
]
#vertical:let,asc horizontal:num,dec
Board4 = [    ['h8', 'h7', 'h6', 'h5', 'h4', 'h3', 'h2', 'h1'],
    ['g8', 'g7', 'g6', 'g5', 'g4', 'g3', 'g2', 'g1'],
    ['f8', 'f7', 'f6', 'f5', 'f4', 'f3', 'f2', 'f1'],
    ['e8', 'e7', 'e6', 'e5', 'e4', 'e3', 'e2', 'e1'],
    ['d8', 'd7', 'd6', 'd5', 'd4', 'd3', 'd2', 'd1'],
    ['c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2', 'c1'],
    ['b8', 'b7', 'b6', 'b5', 'b4', 'b3', 'b2', 'b1'],
    ['a8', 'a7', 'a6', 'a5', 'a4', 'a3', 'a2', 'a1']
]

In [ ]:
import torch
import torch.nn.functional as F

class Model(torch.nn.Module):

    def __init__(self):

        super(Model, self).__init__()

        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(9216, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)

        return output

In [ ]:
ocr = Model()
ocr.load_state_dict(torch.load('/kaggle/input/mn-model/mn.pt'))

In [ ]:
def keep_connected_component(img):
    sharp = sharpen_image(img)
    _, thresholded = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresholded = trim(thresholded)
    thresholded = cv2.bitwise_not(thresholded)
    
    # Detect connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresholded, connectivity=8)
    
    height, width = thresholded.shape
    center = np.array([width // 2, height // 2])
    
    min_distance = float('inf')
    closest_label = None
    
    # Find the component closest to the center (skipping background label 0)
    for i in range(1, num_labels):
        component_centroid = centroids[i]
        distance = np.linalg.norm(component_centroid - center)
        if distance < min_distance:
            min_distance = distance
            closest_label = i
    
    # Create a mask for the closest component
    mask = np.zeros_like(thresholded, dtype=np.uint8)
    
    if closest_label is not None:
        # Mark the closest component in the mask
        mask[labels == closest_label] = 255
        
        # Get the bounding box of the closest component
        x, y, w, h, _ = stats[closest_label]
        
        # Crop the closest component
        cropped_image = thresholded[y:y+h, x:x+w]
    else:
        # If no component found, return the original thresholded image
        cropped_image = thresholded
    
    return cropped_image

In [ ]:
import torch
import cv2

def is_empty(image):
    return image is None or image.size == 0
def is_valid_dict(element, keys):
    return isinstance(element, dict) and all(key in element for key in keys)


def get_grid_trend(warp, grid):
    left = []
    right = []
    top = []
    bottom = []
    
    if isinstance(grid[0][0], dict):
        x1 = int(grid[0][0].get('x', 0) - grid[0][0].get('width', 0) / 2)
        y1 = int(grid[0][0].get('y', 0) - grid[0][0].get('height', 0) / 2)
    else:
        return warp

    required_keys = ['x', 'y', 'width', 'height']
    
    if not is_valid_dict(grid[0][0], required_keys) or not is_valid_dict(grid[-1][-1], required_keys):
        return [[],[]]

    
    

    x1 = int(grid[0][0]['x'] - grid[0][0]['width']/2)
    y1 = int(grid[0][0]['y'] - grid[0][0]['height']/2)
    x2 = int(grid[-1][-1]['x'] + grid[-1][-1]['width']/2)
    y2 = int(grid[-1][-1]['y'] + grid[-1][-1]['height']/2)

    left = warp[y1:y2, :x1, :]
    top = warp[:y1, x1:x2, :]
    right = warp[y1:y2, x2:, :]
    bottom = warp[y2:, x1:x2, :]
    
    if is_empty(left) or is_empty(right) or is_empty(top) or is_empty(bottom):
        return warp  # or return some default value or handle the case as needed

    left = cv2.cvtColor(left, cv2.COLOR_RGB2GRAY)
    right = cv2.cvtColor(right, cv2.COLOR_RGB2GRAY)
    top = cv2.cvtColor(top, cv2.COLOR_RGB2GRAY)
    bottom = cv2.cvtColor(bottom, cv2.COLOR_RGB2GRAY)

    height_vert = int(left.shape[0] / 8)
    height_hor = int(top.shape[1] / 8)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    ocr.to(device)
    ocr.eval()

    num_left = []
    for i in range(8):
     
        each_part = left[i * height_vert: (i + 1) * height_vert,:]
        resized = cv2.resize(each_part, (320, 320), interpolation=cv2.INTER_LINEAR)
        # resized = cv2.bitwise_not(resized)
        # sharp = sharpen_image(resized)

        _, thresholded = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            
        num =  keep_connected_component(resized)
        
       
        
        num = cv2.resize(num, (28, 28), interpolation=cv2.INTER_LINEAR)
        
        
        
        num_tensor = torch.tensor(num, dtype=torch.float32)
        num_tensor = num_tensor.unsqueeze(0).unsqueeze(0) 
        num_tensor = num_tensor / 255.0  
        
        num_tensor = num_tensor.to(device)
        
        with torch.no_grad(): 
            result = ocr(num_tensor)

        probs = torch.softmax(result, dim=1)

        max_prob, preds = torch.max(probs, dim=1)

        num_left.append(preds[0].item())
                
    num_bottom = []
    for i in range(8):
        
        each_part = bottom[:,i * height_hor: (i + 1) * height_hor]
        each_part_rotated = np.rot90(each_part, -1)
        resized = cv2.resize(each_part_rotated, (320, 320), interpolation=cv2.INTER_LINEAR)
        
        _, thresholded = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        num = keep_connected_component(thresholded)
        
        num = cv2.resize(num, (28, 28), interpolation=cv2.INTER_LINEAR)
        num_tensor = torch.tensor(num, dtype=torch.float32)
        num_tensor = num_tensor.unsqueeze(0).unsqueeze(0) 
        num_tensor = num_tensor / 255.0  
        
        contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(thresholded, contours, -1, (0), 1) 
        num_tensor = num_tensor.to(device)

        with torch.no_grad(): 
            result = ocr(num_tensor)

        # Apply softmax to get probabilities
        probs = torch.softmax(result, dim=1)

        # Get the maximum probability and its corresponding index
        max_prob, preds = torch.max(probs, dim=1)

        # Only append the prediction if the probability is greater than 0.5
        num_bottom.append(preds[0].item())
    return [num_left,num_bottom]




In [ ]:
def get_board_information(image):
    board_corners = get_board_corner(image)
    warp, perspective_mat = apply_perspective_transform(image, board_corners)
    inverse_perspective_mat = np.linalg.inv(perspective_mat)
    valid_grid,perspective_grid = get_grid(warp)
    original_grid = transform_to_original_grid(perspective_grid, inverse_perspective_mat)
    return valid_grid,warp,perspective_grid,original_grid,perspective_mat,inverse_perspective_mat

In [ ]:
def map_pieces_to_grid(grid, predictions,labelled_board):

    mapped_pieces = {}

    for piece in predictions:
        piece_x, piece_y = piece["x"], piece["y"]+(piece["height"]/3)
        piece_class = piece["class_name"]
        # Variables to track the closest square
        closest_square = None
        min_distance = float("inf")

        # Iterate through the grid to find the closest square
        for i in range(len(grid)):       # Row index
            for j in range(len(grid[i])):  # Column index
                square_x = grid[i][j]['x']
                square_y = grid[i][j]['y']

                # Calculate Euclidean distance between piece and square center
                distance = euclidean_distance((piece_x, piece_y), (square_x, square_y))
                if distance < min_distance:
                    min_distance = distance
                    closest_square = labelled_board[i][j]  # Convert indices to square name (1-indexed)

        # Map the piece to the closest square
        if closest_square:
            mapped_pieces[closest_square] = piece_class

    return mapped_pieces



In [ ]:
def get_labelled_grid(warp,perspective_grid):
    num_left,num_bottom = get_grid_trend(warp,perspective_grid)[0], get_grid_trend(warp,perspective_grid)[1]

    increase = [1,2,3,4,5,6,7,8]
    decrease = [8,7,6,5,4,3,2,1]
    
    left_increase = sum(1 for i in range(len(num_left)) if num_left[i] == increase[i])
    left_decrease = sum(1 for i in range(len(num_left)) if num_left[i] == decrease[i])
    bottom_decrease = sum(1 for i in range(len(num_bottom)) if num_bottom[i] == decrease[i])
    bottom_increase = sum(1 for i in range(len(num_bottom)) if num_bottom[i] == increase[i])
    
    max_val = max(left_increase, left_decrease, bottom_decrease, bottom_increase)

    print(max_val)
    
    if(max_val ==0 ):
        return Board3
    
    if(max_val == left_increase):
        print("Board3")
        return Board3
    if(max_val == left_decrease):
        print("Board1")
        return Board1
    if(max_val == bottom_increase):
        print("Board2")
        return Board2
    else:
        return Board4

In [ ]:
model = get_model("sp2-ym1iq-9on5y/1", api_key="zZEx3e4qUacIbUbtwsES")

In [ ]:
def rotation_board(frame,labelled_board):
    if (labelled_board == Board1):
        return cv2.rotate(frame, cv2.ROTATE_180)
    elif (labelled_board == Board2):
        return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif (labelled_board == Board4):
        return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        return frame

In [ ]:
def init_frame(frame):
    
    resized_frame = trim(frame)
    valid_board,warp,perspective_grid,original_grid,perspective_mat,inverse_perspective_mat = get_board_information(resized_frame)
    
    if(valid_board):
        labelled_board = get_labelled_grid(warp,perspective_grid)
        resized_frame = rotation_board(resized_frame,labelled_board)
        chess_prediction = get_chess_prediction(resized_frame,model)
        mapped_pieces = map_pieces_to_grid(original_grid, chess_prediction,Board3)
        return mapped_pieces,original_grid,labelled_board
    else:
        print("invalid_board")
        return False,False,False

In [ ]:
def update_frame(frame,labelled_board,original_grid):
    resized_frame = trim(frame)
    resized_frame = rotation_board(resized_frame,labelled_board)
    chess_prediction = get_chess_prediction(resized_frame,model)
    mapped_pieces = map_pieces_to_grid(original_grid, chess_prediction,Board3)
    return mapped_pieces

In [ ]:
class PGNEncoder:
    def __init__(self):
        self.prev_board = {square: None for square in [f"{file}{rank}" for file in "abcdefgh" for rank in "12345678"]}
        self.curr_board = {square: None for square in [f"{file}{rank}" for file in "abcdefgh" for rank in "12345678"]}
        self.pgn_moves = []
        self.Turns = []  # Keeps track of turns (white or black)
        self.black_king_square = ""
        self.white_king_square = ""
        self.endgame = False
    def initial_board(self, board):
        """
        Set up the initial state of the board.
        :param board: A dictionary with keys as square positions and values as piece types (e.g., "white-pawn").
        """
        self.prev_board.update(board)
        for square in board :
            if board[square] == "black-king":
                self.black_king_square = square
                break
        for square in board :
            if board[square] == "white-king":
                self.white_king_square = square

    def update_board(self, board):
        """
        Update the board state with a new board configuration.
        Clears the current board, updates it, and detects moves.
        :param board: A dictionary with keys as square positions and values as piece types (e.g., "white-pawn").
        :return: PGN notation for the detected move or capture.
        """
        # Clear curr_board and update it
        self.curr_board = {square: None for square in self.curr_board}
        self.curr_board.update(board)

        # Detect and record moves
        move = self.detect_move()

        # Determine the turn based on the moved piece

        return move

    def detect_move(self):
        """
        Detect moves or captures based on the difference between prev_board and curr_board.
        :return: PGN notation for the detected move or capture.
        """
        if (self.endgame):
            print("game already end")
            return "valid"
        Missing = []
        Move = []
        Capture = []
        pgn = ""
        for square in self.prev_board:
            prev_piece = self.prev_board[square]
            curr_piece = self.curr_board[square]

            if prev_piece and not curr_piece:
                Missing.append((square, prev_piece))
            elif not prev_piece and curr_piece:
                Move.append((square, curr_piece))
            elif prev_piece and curr_piece and prev_piece != curr_piece:
                Capture.append((square, prev_piece, curr_piece))

        if len(Move) > 1 or len(Missing) > 1:
            print("Ambiguous move detected!")
            return "Ambiguous"
        
        if Missing and Move:
            move_square, curr_piece = Move[0]
            miss_square, miss_piece = Missing[0]
            if miss_piece == curr_piece and miss_square != move_square and self.is_valid_move(curr_piece,miss_square,move_square):
                # Update prev_board
                self.prev_board[miss_square] = None
                self.prev_board[move_square] = curr_piece
                    
                if curr_piece == "black-king" :
                    self.black_king_square = move_square
                elif curr_piece == "white-king" :
                    self.white_king_square = move_square

                # Generate PGN for move
                pgn = self.generate_pgn_move(miss_piece, miss_square, move_square, captured_piece=None)
                self.pgn_moves.append(pgn)

                #Update Turns
                piece_color = "white" if "white" in curr_piece else "black"
                self.Turns.append(piece_color)
                
                

        elif Missing and Capture:
            miss_square, miss_piece = Missing[0]
            capture_square, prev_piece, curr_piece = Capture[0]
            if miss_piece == curr_piece and capture_square != miss_square and self.is_valid_move(curr_piece,miss_square,capture_square):
                
                #Update Turns
                piece_color = "white" if "white" in curr_piece else "black"
                self.Turns.append(piece_color)

                # Update prev_board
                self.prev_board[miss_square] = None
                self.prev_board[capture_square] = curr_piece
                if curr_piece == "black-king" :
                    self.black_king_square = capture_square
                elif curr_piece == "white-king" :
                    self.white_king_square = capture_square 
                # Generate PGN for capture

                pgn = self.generate_pgn_move(miss_piece, miss_square, capture_square, captured_piece=prev_piece)
                self.pgn_moves.append(pgn)
                
        return "valid"
    
    @staticmethod
    def get_piece_notation(piece):
        """
        Returns the PGN notation for the given piece type.
        """
        if "pawn" in piece:
            return ""
        elif "knight" in piece:
            return "N"
        elif "bishop" in piece:
            return "B"
        elif "rook" in piece:
            return "R"
        elif "queen" in piece:
            return "Q"
        elif "king" in piece:
            return "K"
        return ""

    def generate_pgn_move(self, piece, start_square, end_square, captured_piece=None):
        """
        Generate PGN notation for a move or capture.
        :param piece: The piece making the move (e.g., "white-pawn").
        :param start_square: The starting square of the piece (e.g., "d5").
        :param end_square: The destination square of the piece (e.g., "e6").
        :param captured_piece: The captured piece, or None if it's not a capture.
        :return: A string in PGN notation.
        """
        piece_letter = self.get_piece_notation(piece.split("-")[1])
        capture_marker = "x" if captured_piece else ""
        check = self.can_attack_king(end_square)
        check_mate = False
        
        if check :
            if "white" in piece :
                check_mate =  self.check_king_status(self.black_king_square)
            else :
                check_mate =  self.check_king_status(self.white_king_square)
                
        if check and check_mate:
            check_marker = "#"
            self.endgame = True
        elif check:
            check_marker = "+"
        else:
            check_marker = ""

        if "pawn" in piece and captured_piece:
            # For pawn captures, include the file (a-h) of the starting square
            file_from = start_square[0]
            return f"{file_from}{capture_marker}{end_square}{check_marker}"
        elif captured_piece:
            # For captures involving other pieces, include the captured piece's type
            captured_letter = self.get_piece_notation(captured_piece.split("-")[1])
            return f"{piece_letter}{capture_marker}{end_square}{check_marker}"
        else:
            # For non-capture moves
            return f"{piece_letter}{end_square}{check_marker}"
            
    def position_to_coordinates(self,position):
        col, row = position
        return int(row) - 1, ord(col) - ord('a')
        
    # แปลงพิกัด (row, col) กลับเป็นตำแหน่ง เช่น "d4"
    def coordinates_to_position(self,row, col):
        return f"{chr(col + ord('a'))}{row + 1}"
        
    def can_attack_king(self, square):
    
        piece = self.prev_board[square]  # หาหมากในตำแหน่ง square
        if not piece:
            return False  # ไม่มีหมากในตำแหน่งนี้

        # หาตำแหน่งคิงฝั่งตรงข้าม
        if "white" in piece:
            target_king_square = self.black_king_square
        elif "black" in piece:
            target_king_square = self.white_king_square
        else:
            return False  # ไม่ใช่หมากที่รู้จัก

        piece_type = piece.split("-")[1]  # ชนิดหมาก เช่น "queen", "pawn"

        # แปลงตำแหน่งเช่น "d4" เป็นพิกัด (row, col)
        def position_to_coordinates(position):
            col, row = position
            return int(row) - 1, ord(col) - ord('a')

        # แปลงพิกัด (row, col) กลับเป็นตำแหน่ง เช่น "d4"
        def coordinates_to_position(row, col):
            return f"{chr(col + ord('a'))}{row + 1}"

        # ตรวจสอบเส้นทางว่ามีสิ่งกีดขวางหรือไม่
        def is_path_clear(start, end, direction):
            row, col = start
            end_row, end_col = end

            while (row, col) != (end_row, end_col):
                row += direction[0]
                col += direction[1]
                # ตรวจสอบว่าตำแหน่งอยู่ในกระดาน
                if not (0 <= row <= 7 and 0 <= col <= 7):
                    return False  # ถ้าตำแหน่งอยู่นอกกระดาน ให้ถือว่าไม่มีเส้นทาง

                pos = coordinates_to_position(row, col)
                
                if pos != coordinates_to_position(end_row, end_col) and self.prev_board.get(pos) is not None:
                    return False  # มีสิ่งกีดขวาง
            return True

        # แปลงตำแหน่งหมากและคิงเป็นพิกัด
        piece_row, piece_col = position_to_coordinates(square)
        king_row, king_col = position_to_coordinates(target_king_square)
        row_diff, col_diff = king_row - piece_row, king_col - piece_col

        # ตรวจสอบการเดินของหมากแต่ละประเภท
        if piece_type == "queen":
            # Queen: เดินได้ทั้งแนวตรงและแนวทแยง
            if piece_row == king_row or piece_col == king_col:  # แนวตรง
                direction = (0, 1 if row_diff > 0 else -1) if piece_row == king_row else (1 if row_diff > 0 else -1, 0)
                return is_path_clear((piece_row, piece_col), (king_row, king_col), direction)
            if abs(row_diff) == abs(col_diff):  # แนวทแยง
                direction = (1 if row_diff > 0 else -1, 1 if col_diff > 0 else -1)
                return is_path_clear((piece_row, piece_col), (king_row, king_col), direction)

        elif piece_type == "rook":
            # Rook: เดินได้เฉพาะแนวตรง
            if piece_row == king_row or piece_col == king_col:
                direction = (0, 1) if piece_row == king_row else (1 if row_diff > 0 else -1, 0)
                return is_path_clear((piece_row, piece_col), (king_row, king_col), direction)

        elif piece_type == "bishop":
            # Bishop: เดินได้เฉพาะแนวทแยง
            if abs(row_diff) == abs(col_diff):
                direction = (1 if row_diff > 0 else -1, 1 if col_diff > 0 else -1)
                return is_path_clear((piece_row, piece_col), (king_row, king_col), direction)

        elif piece_type == "knight":
            # Knight: เดินเป็นรูปตัว L
            return (abs(row_diff), abs(col_diff)) in [(2, 1), (1, 2)]

        elif piece_type == "pawn":
            # Pawn: เดินเฉพาะแนวเฉียง 1 ช่องเพื่อกินหมาก
            direction = 1 if "white" in piece else -1  # white เดินขึ้น (-1), black เดินลง (+1)
            return (row_diff, col_diff) in [(direction, 1), (direction, -1)]

        elif piece_type == "king":
            # King: เดินได้ทุกทิศทาง 1 ช่อง
            return abs(row_diff) <= 1 and abs(col_diff) <= 1

        return False  # ไม่ใช่หมากที่รู้จักหรือไม่สามารถรุกได้
        
    def check_king_status(self, king_square):
        """
        ตรวจสอบว่าคิงที่ถูกโจมตีสามารถหลบหนีได้หรือไม่
        """
        king_moves = [
            (1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)
        ]
        king_row, king_col = self.position_to_coordinates(king_square)
    
        for move in king_moves:
            new_row, new_col = king_row + move[0], king_col + move[1]
            if 0 <= new_row <= 7 and 0 <= new_col <= 7:  # ตรวจสอบว่าอยู่ในกระดาน
                new_square = self.coordinates_to_position(new_row, new_col)
                if self.prev_board[new_square] is None or self.is_valid_move("king", king_square, new_square):
                    if not any(self.can_attack_king(new_square) for square in self.prev_board):
                        return False  # มีตำแหน่งที่คิงสามารถหนีได้
    
        return True  # คิงไม่มีที่หนี
    
    def is_valid_move(self, piece, start_square, end_square):
        """
        ตรวจสอบว่าการเดินหมากเป็นตาที่ถูกต้องตามกติกาหรือไม่
        Arguments:
            - piece: หมากที่กำลังเดิน (ตัวอย่าง: "white-pawn")
            - start_square: ตำแหน่งเริ่มต้น (ตัวอย่าง: "e2")
            - end_square: ตำแหน่งปลายทาง (ตัวอย่าง: "e4")
        Returns:
            - True: ถ้าการเดินนี้ถูกต้อง
            - False: ถ้าการเดินนี้ไม่ถูกต้อง
        """
        if piece is None or self.prev_board[start_square] != piece:
            return False  # ไม่ใช่ตำแหน่งของหมากที่กำลังเดิน

        # แยกประเภทและสีของหมาก
        piece_type = piece.split("-")[1]
        piece_color = piece.split("-")[0]

        # แปลงตำแหน่ง เช่น "e2" เป็นพิกัด (row, col)
        def position_to_coordinates(position):
            col, row = position
            return int(row) - 1, ord(col) - ord('a')
        
        # แปลงพิกัด (row, col) กลับเป็นตำแหน่ง เช่น "d4"
        def coordinates_to_position(row, col):
            return f"{chr(col + ord('a'))}{row + 1}"
        # ตรวจสอบเส้นทางว่าไม่มีสิ่งกีดขวาง
        def is_path_clear(start, end, direction):
            row, col = start
            end_row, end_col = end

            while (row, col) != (end_row, end_col):
                row += direction[0]
                col += direction[1]
                
                # ตรวจสอบว่าตำแหน่งอยู่ในกระดาน
                if not (0 <= row <= 7 and 0 <= col <= 7):
                    return False  # ถ้าตำแหน่งอยู่นอกกระดาน ให้ถือว่าไม่มีเส้นทาง

                pos = coordinates_to_position(row, col)
                
                if pos != coordinates_to_position(end_row, end_col) and self.prev_board.get(pos) is not None:
                    return False  # มีสิ่งกีดขวาง
            return True


        # ตรวจสอบว่าตำแหน่งอยู่ในกระดาน
        def is_within_board(position):
            row, col = position
            return 0 <= row <= 7 and 0 <= col <= 7

        # ตรวจสอบหมากฝ่ายตรงข้าม
        def is_opponent_piece(piece, target_square):
            target_piece = self.prev_board.get(target_square)
            return target_piece is not None and piece.split("-")[0] != target_piece.split("-")[0]

        # แปลงตำแหน่ง start และ end เป็นพิกัด
        start_row, start_col = position_to_coordinates(start_square)
        end_row, end_col = position_to_coordinates(end_square)
        row_diff, col_diff = end_row - start_row, end_col - start_col

        # ตรวจสอบว่าตำแหน่งอยู่ในกระดาน
        if not is_within_board((start_row, start_col)) or not is_within_board((end_row, end_col)):
            return False

        # ตรวจสอบการเดินสำหรับหมากแต่ละประเภท
        if piece_type == "pawn":
            # return True
            # Pawn: เดินได้ 1 ช่อง (หรือ 2 ช่องถ้าตำแหน่งเริ่มต้น) และกินเฉียง
            direction = 1 if piece_color == "white" else -1  # white เดินขึ้น (+1), black เดินลง (-1)
            if self.prev_board[end_square] is None:  # เดินแบบไม่กิน
                if col_diff == 0 and row_diff == direction:
                    return True
                if col_diff == 0 and row_diff == 2 * direction and 1 <= start_row <= 6:  # เดิน 2 ช่องจากตำแหน่งเริ่มต้น and start_row in (1, 6)
                    return is_path_clear((start_row, start_col), (end_row, end_col), (direction, 0))
            elif is_opponent_piece(piece, end_square):  # เดินแบบกิน
                return (row_diff, col_diff) in [(direction, 1), (direction, -1)]

        elif piece_type == "rook":
            # Rook: เดินเฉพาะแนวตรง
            if start_row == end_row or start_col == end_col:
                direction = (0, 1 if col_diff > 0 else -1) if start_row == end_row else (1 if row_diff > 0 else -1, 0)
                return is_path_clear((start_row, start_col), (end_row, end_col), direction)

        elif piece_type == "bishop":
            # Bishop: เดินเฉพาะแนวทแยง
            if abs(row_diff) == abs(col_diff):
                direction = (1 if row_diff > 0 else -1, 1 if col_diff > 0 else -1)
                return is_path_clear((start_row, start_col), (end_row, end_col), direction)

        elif piece_type == "queen":
            # Queen: เดินได้ทั้งแนวตรงและแนวทแยง
            if start_row == end_row or start_col == end_col:  # แนวตรง
                direction = (0, 1 if col_diff > 0 else -1) if start_row == end_row else (1 if row_diff > 0 else -1, 0)
                return is_path_clear((start_row, start_col), (end_row, end_col), direction)
            if abs(row_diff) == abs(col_diff):  # แนวทแยง
                direction = (1 if row_diff > 0 else -1, 1 if col_diff > 0 else -1)
                return is_path_clear((start_row, start_col), (end_row, end_col), direction)

        elif piece_type == "knight":
            # Knight: เดินเป็นรูปตัว L
            return (abs(row_diff), abs(col_diff)) in [(2, 1), (1, 2)] and (
                self.prev_board[end_square] is None or is_opponent_piece(piece, end_square))

        elif piece_type == "king":
            # King: เดินได้ทุกทิศทาง 1 ช่อง
            return abs(row_diff) <= 1 and abs(col_diff) <= 1 and (
                self.prev_board[end_square] is None or is_opponent_piece(piece, end_square))

        return False  # ไม่ใช่หมากที่รู้จักหรือการเดินไม่ถูกต้อง
        
    def print_pgn_format(self):
        """
        Print the PGN format of the game based on the recorded moves and turns.
        """
        PGN = ""
        current_turn = 1
        
        if len(self.Turns) == 0:
            return PGN
        
        # Start with the first player's move
        FirstPlayer = self.Turns[0]
        
        # Check the first move, if the first player is black, we start with "1...". Otherwise, "1."
        if FirstPlayer != "white":
            PGN = "1. ... "
            current_turn += 1 
        i = 0
        while i < len(self.pgn_moves):
            move = self.pgn_moves[i]
            if i > 1 and i + 2 < len(self.pgn_moves) and self.Turns[i] == self.Turns[i+1] and self.pgn_moves[i][0] ==  self.pgn_moves[i+1][0]:
                print("illegal move")
                i += 2
                continue
                
            # Add moves based on the current player's turn
            if self.Turns[i] == "white":
                PGN += f"{current_turn}. " + move + " "
                current_turn += 1  # Increment turn after the white player's move
            else:
                PGN += move + " "  # Black player's move, no need to add turn number
            i += 1    
        return PGN.strip()


    def end(self):
        """
        End the game and print the PGN format.
        """
        print(self.print_pgn_format())
        


In [ ]:
starter_board = {
    "a1": "white-rook",
    "b1": "white-knight",
    "c1": "white-bishop",
    "d1": "white-queen",
    "e1": "white-king",
    "f1": "white-bishop",
    "g1": "white-knight",
    "h1": "white-rook",

    "a2": "white-pawn",
    "b2": "white-pawn",
    "c2": "white-pawn",
    "d2": "white-pawn",
    "e2": "white-pawn",
    "f2": "white-pawn",
    "g2": "white-pawn",
    "h2": "white-pawn",

    "a7": "black-pawn",
    "b7": "black-pawn",
    "c7": "black-pawn",
    "d7": "black-pawn",
    "e7": "black-pawn",
    "f7": "black-pawn",
    "g7": "black-pawn",
    "h7": "black-pawn",

    "a8": "black-rook",
    "b8": "black-knight",
    "c8": "black-bishop",
    "d8": "black-queen",
    "e8": "black-king",
    "f8": "black-bishop",
    "g8": "black-knight",
    "h8": "black-rook"
}


In [ ]:
import cv2
import os
import csv

# Define video paths directly
video_paths = {
    "2_Move_rotate_student.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/2_Move_rotate_student.mp4",
    "2_move_student.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/2_move_student.mp4",
    "4_Move_studet.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/4_Move_studet.mp4",
    "6_Move_student.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/6_Move_student.mp4",
    "8_Move_student.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos/8_Move_student.mp4",
    "(Bonus)Long_video_student.mp4": "/kaggle/input/cu-chess-detection/Chess Detection Competition/bonus_video/Bonus Long Video Label.mp4"
}

# Define output folder and CSV file
output_folder = "/kaggle/working/"  # Folder for outputs
csv_file = os.path.join(output_folder, "submission.csv")

# Create and initialize the CSV file with default values
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["row_id", "output"])
    # Write initial rows with default output
    for video_name in video_paths.keys():
        writer.writerow([video_name, "1. "])

# Process videos and update the CSV
with open(csv_file, mode='r+', newline='') as file:
    writer = csv.writer(file)
    existing_rows = {row[0]: row[1] for row in csv.reader(file)}  # Track existing rows

    for video_name, video_path in video_paths.items():
        # Default output value
        result = "1. "

        if os.path.exists(video_path):
            print(f"Processing video: {video_name}")

            # Open video file
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                continue
            processed_count = 0
            frame_count = 0
            frame_interval = 50  # Analyze every 100th frame
            start = False
            state = "valid"
            # Initialize the PGN encoder
            pgn_encoder = PGNEncoder()
            Ambiguous_count = 0
            while cap.isOpened():
                if (video_name != "(Bonus)Long_video_student.mp4"):
                    if state == "valid":
                        frame_interval = 50
                    else:
                        frame_interval = 5
                        
                    ret, frame = cap.read()
                    if not ret:
                        # Video ends, finalize the PGN result
                        result = pgn_encoder.print_pgn_format()
                        pgn_encoder.end()
                        break
                    
                    # Process every `frame_interval`-th frame
                    if frame_count % frame_interval == 0:
                        # Chessboard processing
                        if not start:
                            initial_board, original_grid, labelled_board = init_frame(frame)  # Replace with detected board state
                            if initial_board:
                                start = True
                                pgn_encoder.initial_board(initial_board)
                                
                        else:
                            #Detect Hand
                            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                            results = hands.process(frame_rgb)
                            if results.multi_hand_landmarks:
                                print("Hand detected!")
                                state = "invalid"
                                frame_count += 1
                                continue
                                
                            state = "valid"
                            updated_board = update_frame(frame, labelled_board, original_grid)  # Replace with detected board state
                            state = pgn_encoder.update_board(updated_board)
                        processed_count += 1
                    frame_count += 1
                ############################################################################################################################
                else:
                    if state == "valid":
                        print("valid!")
                        frame_interval = 50
                        Ambiguous_count = 0
                    elif state == "Ambiguous" :
                        frame_interval = 10
                        Ambiguous_count += 1
                    else:
                        frame_interval = 10
                    ret, frame = cap.read()    
                    current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)  # Get the current frame index
                    cap.set(cv2.CAP_PROP_POS_FRAMES, current_pos + frame_interval)
                    
                    if not ret:
                        # Video ends, finalize the PGN result
                        result = pgn_encoder.print_pgn_format()
                        pgn_encoder.end()
                        break
                    
                    # Process every `frame_interval`-th frame
                    # Chessboard processing
                    if not start:
                        current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)  # Get the current frame index
                        cap.set(cv2.CAP_PROP_POS_FRAMES, 20)
                        ret, frame = cap.read()
                        initial_board, original_grid, labelled_board = init_frame(frame)  # Replace with detected board state
                        if initial_board:
                            start = True
                            pgn_encoder.initial_board(initial_board)
                           
                                
                    else:
                        #Detect Hand
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        results = hands.process(frame_rgb)
                        if results.multi_hand_landmarks:
                            print("Hand detected!")
                            state = "invalid"
                            frame_count += 1
                            continue
                        
                        if (Ambiguous_count > 70):
                            current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)  # Get the current frame index
                            cap.set(cv2.CAP_PROP_POS_FRAMES, current_pos - 100)
                            ret, frame = cap.read()
                            while True :
                                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                                results = hands.process(frame_rgb)
                                if results.multi_hand_landmarks:
                                    current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)  # Get the current frame index
                                    cap.set(cv2.CAP_PROP_POS_FRAMES, current_pos - 10)
                                    ret, frame = cap.read()
                                    print("Hand detected!")
                                else:
                                    break
                            updated_board = update_frame(frame, labelled_board, original_grid)  # Replace with detected board state
                            pgn_encoder.initial_board(updated_board)
                            state = "valid"
                        else:
                            updated_board = update_frame(frame, labelled_board, original_grid)  # Replace with detected board state
                            state = pgn_encoder.update_board(updated_board)
                    processed_count += 1
                # if (video_name == "(Bonus)Long_video_student.mp4") and processed_count > 200 :
                #     result = pgn_encoder.print_pgn_format()
                #     pgn_encoder.end()
                #     break
            # Release the video file
            cap.release()
            print(f"Finished processing: {video_name}")

        if not result:
            result = "1. "  # Default if no result was generated

        # Update the output value in the CSV
        existing_rows[video_name] = result

    # Rewrite the CSV file with updated results
    file.seek(0)  # Move to the beginning
    for row_id, output in existing_rows.items():
        writer.writerow([row_id, output])

print(f"All videos processed. Results saved in {csv_file}.")